<a href="https://colab.research.google.com/github/joyuno/-/blob/main/Big_Data_Derby_2022(used_car)(%ED%95%84%EC%82%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U autogluon

In [ ]:
# AutoGluon의 TabularPredictor를 임포트하여 자동화된 예측 모델링을 수행할 수 있도록 함
from autogluon.tabular import TabularPredictor

# 경고 메시지를 무시하도록 설정
import warnings
warnings.filterwarnings("ignore")

#pandas와 numpy 임포트
import pandas as pd
import numpy as np

# 시각화 작업을 위한 matplotlib 임포트
import matplotlib.pyplot as plt

# 문자열에서 숫자를 추출하기 위한 정규표현식 임포트
import re

train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

# 추가적인 원본 데이터셋을 불러옴 (중고차 가격 예측 데이터셋)
Original = pd.read_csv('/content/used_cars.csv')
print(Original)
# 'milage'와 'price' 컬럼에 포함된 $빼고 KM빼고 숫자만 추출하여 정수형으로 변환
Original[['milage', 'price']] = Original[['milage', 'price']].map(
    lambda x: int(''.join(re.findall(r'\d+', x))))

# LightGBM, CatBoost, XGBoost 모델을 위한 임포트 (회귀 모델)
import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor

# 난수 생성을 위한 random 임포트
import random

# Support Vector Regression(SVR) 모델을 위한 임포트
from sklearn.svm import SVR

# 모델 성능 평가를 위한 MSE 임포트
from sklearn.metrics import mean_squared_error

# K-fold 교차 검증을 위한 KFold 임포트
from sklearn.model_selection import KFold

# AutoGluon의 TabularPredictor: 모델 예측을 자동화 모델 전처리x 모델 선택 x 알아서 척척
from autogluon.tabular import TabularPredictor

##원본 데이터 original은 실제 데이터이고 train과 test 모두 중고차 가격 예측 데이터 세트에서 훈련된 딥러닝 모델을 통해 생성된  데이터 이다.
# 목적은 원본데이터를 같이 넢고 학습했을때 성능이 향상되는지이다.

In [ ]:
print(train.head(3))
print(train.info())
print(test.info())

In [ ]:
train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True) # 필요없는 칼럼 제거


train = pd.concat([train, Original], ignore_index=True) # [train,original] 결합

In [ ]:
## feature engineering
# age 피처 더 명확하게 살리기
def extract_age_features(df):
    current_year = 2024 # 현재 년도

    # Vehicle_Age: 차량 연령 계산, 년도와 같은 범주형 변수를 수치화 시킴
    df['Vehicle_Age'] = current_year - df['model_year']

    # Mileage_per_Year: 연간 주행 거리 계산
    df['Mileage_per_Year'] = df['milage'] / df['Vehicle_Age']

    # milage_with_age: 같은 연식 차량들의 평균 마일리지 계산
    df['milage_with_age'] = df.groupby('Vehicle_Age')['milage'].transform('mean')
    #df.groupby('Vehicle_Age')['milage'].mean()은 각 Vehicle_Age 값에 대해 평균 마일리지만 계산
    # 그러나 이 결과는 원본 데이터와 일치하지 않기 때문에
    # 마일리지 값을 각 행에 맞게 다시 분배하려면 transform
    # transform은 각 그룹에 대해 함수를 적용한 후 결과를 원본 데이터의 각 행에 맞게 반복해서 반환
    # Mileage_per_Year_with_age: 같은 연식 차량들의 평균 연간 주행 거리 계산
    df['Mileage_per_Year_with_age'] = df.groupby('Vehicle_Age')['Mileage_per_Year'].transform('mean')

    return df
# 브랜드 피처 살리기
def extract_other_features(df):
    # 고급 브랜드 리스트
    luxury_brands = ['Mercedes-Benz', 'BMW', 'Audi', 'Porsche', 'Land',
                    'Lexus', 'Jaguar', 'Bentley', 'Maserati', 'Lamborghini',
                    'Rolls-Royce', 'Ferrari', 'McLaren', 'Aston', 'Maybach']

    # 브랜드가 고급 브랜드 리스트에 포함되면 1, 아니면 0
    df['Is_Luxury_Brand'] = df['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

    return df
# 함수 적용
train = extract_age_features(train)
test = extract_age_features(test)

train = extract_other_features(train)
test = extract_other_features(test)


In [ ]:

def update(df):

    t = 100  # 기준값, 조건에 맞는 값은 "noise"로 대체
    # 범주형 변수 리스트
    cat_c = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
    # 빈도수를 기준으로 대체
    re_ = ['model', 'engine', 'transmission', 'ext_col', 'int_col']

    # re_에 포함된 열의 빈도수(value_count) series 모형에서 100미만 값들을 noise로 대체
    for col in re_:
        df.loc[df[col].value_counts(dropna=False)[df[col]].values < t, col] = "noise"
    # na 값은 missing으로 이후 타입을 category로 변경
    for col in cat_c:
        df[col] = df[col].fillna('missing')
        df[col] = df[col].astype('category')

    return df

train  = update(train)
test   = update(test)
#각 행마다 해당 값의 출현 빈도를 기준으로 새로운 특성을 생성할 때 사용.
print(train['model'].value_counts(dropna=False)[train['model']])
X = train.drop('price', axis=1)
y = train['price']

In [ ]:
print(train.shape)

In [ ]:
#그냥 특정 열의 값 빈도수만 계산
print(train['model'].value_counts(dropna=False))

In [ ]:
import numpy as np
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold


# LightGBM의 콜백 함수 설정 (학습 로깅과 조기 종료 조건)
callbacks = [log_evaluation(period=300), early_stopping(stopping_rounds=200)] # 300번마다 학습을 주기적으로 체크하겠다 200번동안 개선되지않으면 중단하겠다
# 모든 범주형 열 리스트화
cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"cat_cols--------{cat_cols}")

# 교차 검증과 OOF 예측을 통해 MAE 또는 MSE를 계산하는 함수 정의
def get_MAE_oof(df, target, lgb_params, cat_params=None, model_type='LGBM'):

    # 초기 설정
    oof_predictions = np.zeros(len(df))
    kf = KFold(n_splits=5, shuffle=True, random_state=1)  # 5개의 Fold로 분할
    models = [] # 모델을 저장할 리스트
    rmse_scores = [] # RMSE 점수를 저장할 리스트
    # K-Fold 교차 검증
    for fold, (train_idx, val_idx) in enumerate(kf.split(df)): # kf.split()을 쓰면 몇번 fold인지와 뭘 쪼갰는지train, val의 각인덱스를 출력한다

        print(f"Training fold {fold + 1}/{5} with {model_type}")
        # 훈련/검증 데이터 분리
        X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
        y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]
        # LightGBM
        if model_type == 'LGBM':
            train_data = lgb.Dataset(X_train, label=y_train) #Dataet 메소드
            val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

            model = lgb.train(
                lgb_params,
                train_data,
                valid_sets=[train_data, val_data],
                valid_names=['train', 'valid'],
                callbacks=callbacks
            )
        # CatBoost # 범주형 변수 특화모델 Pool 이라는 객체가 범주형 변수를 자동으로 최적화 처리
        elif model_type == 'CAT':
            train_data = Pool(data=X_train, label=y_train , cat_features=cat_cols) #범주형 변수 이름이나 인덱스를 위 모든 범주형 변수 리스트로 지정
            val_data = Pool(data=X_val, label=y_val , cat_features=cat_cols)

            model = CatBoostRegressor(**cat_params)#unpacking 딕셔너리 형태로 저장한 파라미터를 언패킹하여 보다 간결한 코드로 전달
            model.fit(train_data, eval_set=val_data, verbose=150, early_stopping_rounds=200) #log_evaluation는 훈련손실 검증손실과 verbose는 손실함수 기록
        #두 모델 같이 저장 앙상블
        models.append(model)
        # 검증 데이터 예측
        if model_type == 'LGBM':
            pred = model.predict(X_val, num_iteration=model.best_iteration)# 최고성능을 낸 반복 early stopping과 함께사용해야한다
        elif model_type == 'CAT':
            pred = model.predict(X_val)

        rmse = np.sqrt(mean_squared_error(y_val, pred))
        rmse_scores.append(rmse)

        print(f'{model_type} Fold RMSE: {rmse}')

        oof_predictions[val_idx] = pred #Out-of-Fold 예측이라고 하며 각 폴드별로 train,val을 나눴으면 예시 폴드 5개이며 train 9개 val1개로 나눴을때 5폴드로나온 5개의 val가 oof_prediction에 val_idx에 들어가서

    print(f'Mean RMSE: {np.mean(rmse_scores)}')
    return oof_predictions, models




lgb_params = {
    'objective': 'MAE',
    'n_estimators': 1000,
    'random_state': 1,
}

oof_predictions_lgbm, models_lgbm = get_MAE_oof(X, y, lgb_params, model_type='LGBM')
X['LGBM_MAE'] = oof_predictions_lgbm


LGBM_preds = np.zeros(len(test))
for model in models_lgbm:#각 모델의 예측값을 모델 개수로 나누어 평균을 구합니다.
# 이렇게 함으로써 여러 앙상블 모델의 예측 결과의 평균을 구함으로써 보다안정된 결과를 얻을수잇음
    LGBM_preds += model.predict(test) / len(models_lgbm)
test['LGBM_MAE'] = LGBM_preds



lgb_params = {
    'objective': 'MSE',
    'n_estimators': 1000,
    'random_state': 1,
}

oof_predictions_lgbm, models_lgbm = get_MAE_oof(X, y, lgb_params, model_type='LGBM')

X['LGBM_MSE_diff'] = oof_predictions_lgbm - X['LGBM_MAE']


LGBM_preds = np.zeros(len(test))
for model in models_lgbm:
    LGBM_preds += model.predict(test) / len(models_lgbm)
test['LGBM_MSE_diff'] = LGBM_preds - test['LGBM_MAE']

test.head()

In [ ]:
X['price'] = y
#이게 autogluon 알아서 뚝딱딱
predictor = TabularPredictor(label='price',
                            eval_metric='rmse',
                            problem_type='regression').fit(X,
                                                       presets='best_quality',
                                                       time_limit=3600*1,
                                                       verbosity=2,
                                                       num_gpus=0,
                                                       included_model_types=['GBM', 'CAT']
                                                      )
y_pred = predictor.predict(test)

# A bit of blending with the solution already blended from kagglers. Can be tweaked (currently 50/50).
sub_blend = pd.read_csv('/kaggle/input/top-5-blended-car-prices/submission_9.csv')
sample_sub = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
sample_sub['price'] =  y_pred * 0.55 + sub_blend['price'] * 0.45
sample_sub.to_csv("submission.csv", index=False)
sample_sub.head()
#Autogluon 참고 링크
#https://familia-89.tistory.com/77
